In [ ]:
# prompt design in LLMCRS

prompt_subtask_detection = f'''The AI assistant should analyze the dialogue context to detect which sub-task should be selected. The sub- tasks include: { subtask_lists }. The selected sub-task need to represent by its corresponding sub-task schema. The sub-task schema is { subtask_schema_json }. There are some cases for your reference: { Demonstrations }. The dialogue context is { Dialogue_Context }. From this dialogue context, which sub-task should be selected? The sub-task MUST be selected from the above list and represented by the schema.'''
prompt_model_matching = f'''The AI assistant should select the most appropriate expert model to process the sub-task based on the sub- task goal and expert model description. The sub-task goal is { SubTask_Goal }. The list of expert models is { [ID, Description] }. Please select one expert model. The expert model MUST be selected from the above list and represented by the ID.'''
prompt_response_generation = f'''With the dialogue context and the sub-task results, the AI assistant needs to generate a response to the user. The dialogue context is { Dialogue_Context }. The sub-task results can be formed as: Sub-Task Name: { SubTask_Name }, Expert Model: { ID, Description }, and Sub- Task Output: { Output }. Please generate a response to answer the user’s request.'''

## 1) schema instruction

- subtask manager의 역할
- subtask 종류
- subtask 스키마

In [35]:
subtask_lists = '[ User preference elicitation, Recommendation, Explanation, Item detail search ]'

subtask_schema_table = '''
1) Sub-task name: It provides a unique identifier for sub-task detection, which is used for references to related expert models and their predicted results.
2) Sub-task arguments: It contains the list of required arguments for sub-task execution. They are resolved from either the dia- logue context or the predicted results of the sub-tasks executed in the previous dialogue turns.
3) Output type: It indicates the type of the outputs of the sub-task.

| Sub Task Name | Sub-Task Arguments | Output type |
| ———————— | ——————————- | ——————- |
| User preference elicitation | {category, dialogue context} | Text |
| Recommendation | {category, dialogue context} | Item |
| Explanation | {item name, dialogue context} | Text |
| Item detail search | {item name, item attribute} | Text |
'''

subtask_schema_json = """1) Sub-task name: It provides a unique identifier for sub-task detection, which is used for references to related expert models and their predicted results.
2) Sub-task arguments: It contains the list of required arguments for sub-task execution. They are resolved from either the dia- logue context or the predicted results of the sub-tasks executed in the previous dialogue turns.
3) Output type: It indicates the type of the outputs of the sub-task.

[{'subtask name': 'user preference elicitation', 'subtask arguments': ['category', 'dialogue context'], 'output type':'text'},
{'subtask name': 'recommendation', 'subtask arguments': ['category','dialogue context'], 'output type': 'item'},
{'subtask name': 'explanation', 'subtask arguements': ['item name', 'dialogue context'], 'output type': 'text'},
{'subtask name': 'item detail search', 'subtask arguments': ['item name', 'item attribute'], 'output type': 'text'}]"""

In [ ]:
demonstraion_instruction = '''user_query: I want to find a legal drama
answer: 
    subtask name: user preference elicitation,
    subtask arguments: ["legal drama", "I want to find a legal drama"],
    output type: text

user_query: I want the female actor as the lead role
answer:
    "subtask name": "recommendation",
    "subtask arguments": ["legal drama", "I want to find a legal drama\nI want the female actor as the lead role"],
    "output type": "item"

user_query: Why do you recommend it to me?
answer:
    "subtask name": "explanation",
    "subtask arguments": ["Suits", "I want to find a legal drama\nI want the female actor as the lead role"],
    "output type": "text"

user_query: OK. what timeframe is it?
answer:
    "subtask name": "item detail research",
    "subtask arguments": ["Suits", "I want to find a legal drama\nI want the female actor as the lead role\nOK. Why do you recommend it to me?\nWhat timeframe is it?"],
    "output type": "item"
'''

In [96]:
subtask_detection_prompt = '''The AI assistant should analyze the dialogue context to detect which sub-task should be selected.
The subtasks include: {subtask_lists}.
The subtask schema is {subtask_schema}.
The selected sub-task need to represent by its corresponding sub-task schema.

There are some cases for your reference.

user_query: I want to find a legal drama
answer: 
    subtask name: user preference elicitation,
    subtask arguments: ["legal drama", "I want to find a legal drama"],
    output type: text

user_query: I want the female actor as the lead role
answer:
    "subtask name": "recommendation",
    "subtask arguments": ["legal drama", "I want to find a legal drama\nI want the female actor as the lead role"],
    "output type": "item"

user_query: Why do you recommend it to me?
answer:
    "subtask name": "explanation",
    "subtask arguments": ["Suits", "I want to find a legal drama\nI want the female actor as the lead role"],
    "output type": "text"

user_query: OK. what timeframe is it?
answer:
    "subtask name": "item detail research",
    "subtask arguments": ["Suits", "I want to find a legal drama\nI want the female actor as the lead role\nOK. Why do you recommend it to me?\nWhat timeframe is it?"],
    "output type": "item"

Which sub-task should be selected? The sub-task MUST be selected from the above list and represented by the schema.
user_query: {query}
answer:
'''


In [103]:
from langchain import LLMChain, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain


manager = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0
)

subtask_detection = PromptTemplate(
    template  = subtask_detection_prompt ,
    input_variables = ['subtask_lists', 'subtask_schema', 'query']
)


# Chain 정의하기
chain = LLMChain(
    llm = manager,
    prompt = subtask_detection
)


result = chain.invoke({'subtask_lists': subtask_lists, 'subtask_schema': subtask_schema_json, 'query': 'I need movies with hell lot of dopamine'}) # chain 실행
print(result)

{'subtask_lists': '[ User preference elicitation, Recommendation, Explanation, Item detail search ]', 'subtask_schema': "1) Sub-task name: It provides a unique identifier for sub-task detection, which is used for references to related expert models and their predicted results.\n2) Sub-task arguments: It contains the list of required arguments for sub-task execution. They are resolved from either the dia- logue context or the predicted results of the sub-tasks executed in the previous dialogue turns.\n3) Output type: It indicates the type of the outputs of the sub-task.\n\n[{'subtask name': 'user preference elicitation', 'subtask arguments': ['category', 'dialogue context'], 'output type':'text'},\n{'subtask name': 'recommendation', 'subtask arguments': ['category','dialogue context'], 'output type': 'item'},\n{'subtask name': 'explanation', 'subtask arguements': ['item name', 'dialogue context'], 'output type': 'text'},\n{'subtask name': 'item detail search', 'subtask arguments': ['ite

In [104]:
print(result['text'])

{
    "subtask name": "user preference elicitation",
    "subtask arguments": ["dopamine", "I need movies with hell lot of dopamine"],
    "output type": "text"
}


In [101]:
system_prompt = '''The AI assistant should analyze the dialogue context to detect which sub-task should be selected.
The subtasks include: {subtask_lists}.
The subtask schema is {subtask_schema}.
The selected sub-task need to represent by its corresponding sub-task schema.

There are some cases for your reference.

user_query: I want to find a legal drama
answer: 
    subtask name: user preference elicitation,
    subtask arguments: ["legal drama", "I want to find a legal drama"],
    output type: text

user_query: I want the female actor as the lead role
answer:
    "subtask name": "recommendation",
    "subtask arguments": ["legal drama", "I want to find a legal drama\nI want the female actor as the lead role"],
    "output type": "item"

user_query: Why do you recommend it to me?
answer:
    "subtask name": "explanation",
    "subtask arguments": ["Suits", "I want to find a legal drama\nI want the female actor as the lead role"],
    "output type": "text"

user_query: OK. what timeframe is it?
answer:
    "subtask name": "item detail research",
    "subtask arguments": ["Suits", "I want to find a legal drama\nI want the female actor as the lead role\nOK. Why do you recommend it to me?\nWhat timeframe is it?"],
    "output type": "item"

Which sub-task should be selected? The sub-task MUST be selected from the above list and represented by the schema.'''

user_messsage = '''
user_query: {query}
answer:'''

In [102]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="user_query: I need movies with hell lot of dopamine\nanswer:"
            )
        ],
        'subtask_lists': subtask_lists,
        'subtask_schema': subtask_schema_json,
    }
)

AIMessage(content='{\n    "subtask name": "user preference elicitation",\n    "subtask arguments": ["movies with high dopamine", "I need movies with hell lot of dopamine"],\n    "output type": "text"\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 572, 'total_tokens': 616}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-bdf1a9a9-e555-4e08-9776-0de7f458ff51-0', usage_metadata={'input_tokens': 572, 'output_tokens': 44, 'total_tokens': 616})

## 2. Expert Model Lists

In [97]:
expert_lists = "[ ‘BERT’, ‘GPT-2’, ‘ReDial’, ‘KBRD’, ‘KGSF’, ‘TG-ReDial’ ]"